In [62]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

print("Its alive!")

Its alive!


In [16]:
#Functions


In [53]:
drop_list = []
df = pd.read_csv("./Utah_Crash_Data_2020.csv")
df = df.drop([
"CRASH_ID",
# "CRASH_DATETIME",
"CRASH_SEVERITY_ID",
"ROUTE",
"MILEPOINT",
"LAT_UTM_Y",
"LONG_UTM_X",
"MAIN_ROAD_NAME",  
"CITY",                              
"COUNTY_NAME",                       
"WORK_ZONE_RELATED",                 
"PEDESTRIAN_INVOLVED",               
"BICYCLIST_INVOLVED",                
"MOTORCYCLE_INVOLVED",               
"IMPROPER_RESTRAINT",                
"UNRESTRAINED",                      
"DUI",                               
"INTERSECTION_RELATED",              
"WILD_ANIMAL_RELATED",               
"DOMESTIC_ANIMAL_RELATED",           
"OVERTURN_ROLLOVER",                 
"COMMERCIAL_MOTOR_VEH_INVOLVED",     
"TEENAGE_DRIVER_INVOLVED",           
"OLDER_DRIVER_INVOLVED",             
"NIGHT_DARK_CONDITION",              
"SINGLE_VEHICLE",                    
"DISTRACTED_DRIVING",                
"DROWSY_DRIVING",                   
"ROADWAY_DEPARTURE"
], axis=1)
df = df.dropna()


C:\Users\whyri\AppData\Local\Temp\ipykernel_8024\4135842229.py:2: DtypeWarning: Columns (0,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Utah_Crash_Data_2020.csv")


In [54]:
#Takes around 2:30 min to run
df["CRASH_MONTH"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.month
df["CRASH_WEEKDAY"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.day_name()
df["CRASH_HOUR"] = pd.to_datetime(df["CRASH_DATETIME"]).dt.hour

df = df.drop(columns=["CRASH_DATETIME"])
df

,CRASH_MONTH,CRASH_WEEKDAY,CRASH_HOUR
0,2,Friday,10
1,12,Tuesday,12
2,12,Saturday,20
3,12,Saturday,0
4,12,Wednesday,17
...,...,...,...
252495,11,Thursday,23
252496,10,Tuesday,19
252497,10,Friday,17
252498,11,Wednesday,18


In [55]:
df_agg = df.groupby(['CRASH_MONTH', 'CRASH_WEEKDAY','CRASH_HOUR']).size().reset_index(name="COUNT")
#Although the below columns are numeric, they are categorical because they are a time not a measurement
df_agg = pd.get_dummies(df_agg, columns=["CRASH_MONTH","CRASH_WEEKDAY","CRASH_HOUR"])
df_agg.set_index("COUNT")

,CRASH_MONTH_1,CRASH_MONTH_2,CRASH_MONTH_3,CRASH_MONTH_4,CRASH_MONTH_5,CRASH_MONTH_6,CRASH_MONTH_7,CRASH_MONTH_8,CRASH_MONTH_9,CRASH_MONTH_10,...,CRASH_HOUR_14,CRASH_HOUR_15,CRASH_HOUR_16,CRASH_HOUR_17,CRASH_HOUR_18,CRASH_HOUR_19,CRASH_HOUR_20,CRASH_HOUR_21,CRASH_HOUR_22,CRASH_HOUR_23
COUNT,,,,,,,,,,,,,,,,,,,,,
54,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
38,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
29,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
34,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
27,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
109,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
62,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [63]:
# Split dataset in features and target variable

y = df_agg["COUNT"] # Label
X = df_agg.drop(columns=["COUNT"]) # Features
X.head()

,CRASH_MONTH_1,CRASH_MONTH_2,CRASH_MONTH_3,CRASH_MONTH_4,CRASH_MONTH_5,CRASH_MONTH_6,CRASH_MONTH_7,CRASH_MONTH_8,CRASH_MONTH_9,CRASH_MONTH_10,...,CRASH_HOUR_14,CRASH_HOUR_15,CRASH_HOUR_16,CRASH_HOUR_17,CRASH_HOUR_18,CRASH_HOUR_19,CRASH_HOUR_20,CRASH_HOUR_21,CRASH_HOUR_22,CRASH_HOUR_23
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [64]:
# Set label and features
y = df_agg["COUNT"] # Label
X = df_agg.drop(columns=["COUNT"]) # Features

# Run the multiple linear regression model
model = sm.OLS(y, X).fit()

# View results
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                  COUNT   R-squared:                       0.794
Model:                            OLS   Adj. R-squared:                  0.789
Method:                 Least Squares   F-statistic:                     189.7
Date:                Tue, 05 Apr 2022   Prob (F-statistic):               0.00
Time:                        18:59:58   Log-Likelihood:                -10369.
No. Observations:                2016   AIC:                         2.082e+04
Df Residuals:                    1975   BIC:                         2.105e+04
Df Model:                          40                                         
Covariance Type:            nonrobust                                         
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
CRASH_MONTH_1             

In [65]:
df_insample = pd.DataFrame({'Actual':df_agg['COUNT'], 
                    'Predicted':model.fittedvalues, 
                    'Residuals (Error)':df_agg['COUNT'] - model.fittedvalues})

df_insample.head(10)

,Actual,Predicted,Residuals (Error)
0,54,86.324219,-32.324219
1,38,64.187500,-26.187500
2,29,58.324219,-29.324219
3,34,54.785156,-20.785156
4,27,57.894531,-30.894531
5,65,79.753906,-14.753906
6,154,120.574219,33.425781
7,252,197.996094,54.003906
8,269,211.812500,57.187500
9,203,166.664062,36.335938


In [67]:
def regression_fit(model, y_actual):

    y_pred = model.fittedvalues

    print(f"R2:\t{round(model.rsquared, 4)}")
    print(f"R2-adj:\t{round(model.rsquared_adj, 4)}")
    print(f"MAE:\t{round(abs(y_pred - y_actual).mean(), 4)}")
    print(f"RMSE:\t{round(((y_pred - y_actual)**2).mean() ** (1/2), 4)}")

regression_fit(model, y)

R2:	0.7935
R2-adj:	0.7893
MAE:	30.9245
RMSE:	41.4398


In [73]:
print("MAX: ", df_agg['COUNT'].max())
print("MIN: ", df_agg['COUNT'].min())
print("MEAN: ", df_agg['COUNT'].mean())
print("MEDIAN: ", df_agg['COUNT'].median())

MAX:  535
MIN:  4
MEAN:  125.24702380952381
MEDIAN:  110.0
